In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run

import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [2]:
# Example Bloom Filter

from helper import hash256

bit_field_size = 10
bit_field = [0] * bit_field_size

h256 = hash256(b'hello world')
bit = int.from_bytes(h256, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [3]:
# Example Bloom Filter 2

from helper import hash256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = hash256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
h = hash256(b'goodbye')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [4]:
# Example Bloom Filter 3

from helper import hash160, hash256

bit_field_size = 10
bit_field = [0] * bit_field_size

phrase1 = b'hello world'
h1 = hash256(phrase1)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase1)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
phrase2 = b'goodbye'
h1 = hash256(phrase2)
bit1 = int.from_bytes(h1, 'big') % bit_field_size
bit_field[bit1] = 1
h2 = hash160(phrase2)
bit2 = int.from_bytes(h2, 'big') % bit_field_size
bit_field[bit2] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [5]:
# Example BIP0037 Bloom Filter

from helper import murmur3
from bloomfilter import BIP37_CONSTANT

field_size = 2
num_functions = 2
tweak = 42

bit_field_size = field_size * 8
bit_field = [0] * bit_field_size

for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 1

#### 1.1 Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`

#### 1.2. Make [this test](/edit/session8/bloomfilter.py) pass.

In [6]:
# Exercise 1.1
from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')

# bit_field_size is 8 * field_size
bit_field_size = field_size * 8
# create a bit field with the appropriate size
bit_field = [0] * bit_field_size

# for each item you want to add to the filter
for item in items:
    # iterate function_count number of times
    for i in range(function_count):
        # BIP0037 spec seed is i*BIP37_CONSTANT + tweak
        seed = i * BIP37_CONSTANT + tweak
        # get the murmur3 hash given that seed
        h = murmur3(item, seed=seed)
        # set the bit to be h mod the bit_field_size
        bit = h % bit_field_size
        # set the bit_field at the index bit to be 1
        bit_field[bit] = 1
# print the bit field converted to bytes using bit_field_to_bytes in hex
print(bit_field_to_bytes(bit_field).hex())

4000600a080000010940


In [7]:
# Exercise 1.2

reload(bloomfilter)
run(bloomfilter.BloomFilterTest('test_add'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 2

#### 2.1. Make [this test](/edit/session8/bloomfilter.py) pass.

#### 2.2. Do the following:

* Connect to a testnet node
* Load a filter for your testnet address
* Send a request for transactions from the block which had your previous testnet transaction
* Receive the merkleblock and tx messages.

In [8]:
# Exercise 2.1

reload(bloomfilter)
run(bloomfilter.BloomFilterTest('test_filterload'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [9]:
# Exercise 2.2

from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from merkleblock import MerkleBlock
from network import (
    FILTERED_BLOCK_DATA_TYPE, 
    GetDataMessage,
    SimpleNode,
)

block_hash = bytes.fromhex('00000000000001fd16e986436252c023b2f9ba729319309189af9ab5be9d4ff9')
passphrase = b'jimmy@programmingblockchain.com secret passphrase'  # FILL THIS IN
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
filter_size = 30
filter_num_functions = 5
filter_tweak = 90210  # FILL THIS IN

# get the hash160 of the address, which is what we need to add to our filter
h160 = decode_base58(addr)
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
bf = BloomFilter(filter_size, filter_num_functions, filter_tweak)
# add the h160 to the bloom filter
bf.add(h160)

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)
# complete the handshake
node.handshake()
# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())
# create a getdata message
getdata = GetDataMessage()
# add a FILTERED_BLOCK_DATA_TYPE for the block hash above
getdata.add_data(FILTERED_BLOCK_DATA_TYPE, block_hash)
# send a getdata message for a FILTERED_BLOCK_DATA_TYPE
node.send(getdata.command, getdata.serialize())
# wait for the merkleblock command
envelope = node.wait_for_commands([b'merkleblock'])
# wait for the tx command
envelope = node.wait_for_commands([b'tx'])
# print the payload in hex
print(envelope.payload.hex())

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
sending: version: 7f1101000000000000000000ad02e65b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d20b5fd779e06cd5cc31b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d04000000000000af02e65b00000000000000000000000000000000000000000000ffff4830fd33d9640d04000000000000000000000000000000000000000000000000a2efd63bd7809e85102f5361746f7368693a302e31372e302fa103160001
sending: verack: 
receiving: verack: 
sending: filterload: 1e000000000000008800000002000000000000000000200000000100000000050000006260010001
sending: getdata: 0103000000f94f9dbeb59aaf899130199372baf9b223c052624386e916fd01000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: edf29c6b524abe9e
sending: pong: edf29c6b524abe9e
receiving: addr: 01af02e65b0d0400000000000000000000000000000000ffffa2d49eac479d
receiving

### Exercise 3

#### 3.1. Make [this test](/edit/session8/merkleblock.py) pass.

In [10]:
# Exercise 3.1

reload(merkleblock)
run(merkleblock.MerkleBlockTest('test_is_valid'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 4

#### 4.1. You have been sent some unknown amount of testnet bitcoins to your address. 

Send all of it back (minus fees) to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` using only the networking protocol.

In [11]:
# Exercise 4.1

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import (
    decode_base58,
    hash256,
    encode_varint,
    little_endian_to_int, 
    read_varint,
    SIGHASH_ALL,
)
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script
from tx import Tx, TxIn, TxOut

last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
passphrase = b'Jimmy Song Programming Blockchain'  # FILL THIS IN
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
print(addr)
h160 = decode_base58(addr)
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
filter_size = 30
filter_num_functions = 5
filter_tweak = 90210  # FILL THIS IN
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis

# connect to tbtc.programmingblockchain.com in testnet mode, logging True
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
bf = BloomFilter(filter_size, filter_num_functions, filter_tweak)
# add the h160 to the bloom filter
bf.add(h160)
# complete the handshake
node.handshake()
# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())

# set start block to last_block from above
start_block = bytes.fromhex(last_block_hex)
# create GetHeadersMessage with this start block
getheaders_message = GetHeadersMessage(start_block=start_block)
# send a getheaders message with the start block
node.send(getheaders_message.command, getheaders_message.serialize())

# wait for the headers message
headers_envelope = node.wait_for_commands({HeadersMessage.command})
# get the stream from the headers
stream = headers_envelope.stream()
# parse the headers message
headers = HeadersMessage.parse(stream)

# store the last block as None
last_block = None
# initialize the GetDataMessage
get_data_message = GetDataMessage()
# loop through the blocks in the headers
for block in headers.blocks:
    # check that the proof of work on the block is valid
    if not block.check_pow():
        raise RuntimeError
    # check that this block's prev_block is the last block
    if last_block is not None and block.prev_block != last_block:
        raise RuntimeError
    # set the last block to the current hash
    last_block = block.hash()
    # add FILTERED_BLOCK_DATA_TYPE last_block to GetDataMessage
    get_data_message.add_data(FILTERED_BLOCK_DATA_TYPE, last_block)
# send the get_data_message
node.send(get_data_message.command, get_data_message.serialize())

# initialize prev_tx, prev_index and prev_tx_obj to None
prev_tx, prev_index, prev_tx_obj = None, None, None
# while prev_tx is None 
while prev_tx is None:
    # wait for the merkleblock or tx commands
    envelope = node.wait_for_commands([b'merkleblock', b'tx'])
    # initialize the stream from the envelope
    stream = envelope.stream()
    # if we have the merkleblock command
    if envelope.command == b'merkleblock':
        # parse the MerkleBlock
        mb = MerkleBlock.parse(stream)
        # check that the MerkleBlock is valid
        if not mb.is_valid():
            raise RuntimeError
    # else we have the tx command
    else:
        # parse the tx
        prev_tx_obj = Tx.parse(stream, testnet=True)
        # loop through the enumerated tx outs (enumerate(t.tx_outs))
        for i, tx_out in enumerate(prev_tx_obj.tx_outs):
            print(tx_out)
            # if our output has the same address as our address (addr) we found it
            if tx_out.script_pubkey.address(testnet=True) == addr:
                # we found our utxo. set prev_tx, prev_index, prev_amount
                prev_tx = prev_tx_obj.hash()
                prev_index = i
                prev_amount = tx_out.amount
# create tx_in
tx_in = TxIn(prev_tx, i)
# get the previous amount from prev_tx_obj
prev_amount = prev_tx_obj.tx_outs[prev_index].amount
# calculate the output amount (prev_amount - fee)
output_amount = prev_amount - fee
# create tx_out
tx_out = TxOut(output_amount, target_script)
# create transaction on testnet
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# sign the one input we have
tx_obj.sign_input(0, private_key)
# serialize and hex to see what it looks like
print(tx_obj.serialize().hex())
# send this signed transaction on the network
node.send(b'tx', tx_obj.serialize())
# wait a sec so this message goes through to the other node time.sleep(1) 
time.sleep(1)
# now ask for this transaction from the other node
# create a GetDataMessage
getdata = GetDataMessage()
# ask for our transaction by adding its hash and TX_DATATYPE to the message
getdata.add_data(TX_DATA_TYPE, tx_obj.hash())
# send the message
node.send(getdata.command, getdata.serialize())
# now wait for a response
envelope = node.wait_for_commands({b'tx', b'reject'})
if envelope.command == b'tx':
    print('success!')
    print(tx_obj.serialize().hex())
    print(envelope.payload.hex())
    print(tx_obj.hash().hex())
else:
    print(envelope.payload)

mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv
sending: version: 7f1101000000000000000000ad02e65b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d2007c82899d9c9ba461b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
receiving: version: 7f1101000d04000000000000af02e65b00000000000000000000000000000000000000000000ffff4830fd33d9650d04000000000000000000000000000000000000000000000000e81da573bea6cc5d102f5361746f7368693a302e31372e302fa103160001
sending: verack: 
receiving: verack: 
sending: filterload: 1e000000000448000000000004000000000200000000000000000000000000050000006260010001
sending: getheaders: 7f11010001192e91ea55ec59872747ba44fb1a5556499a4f64366324c2d5380500000000000000000000000000000000000000000000000000000000000000000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: a2ccebb38321ce8a
sending: pong: a2ccebb38321ce8a
receiving: addr: 01af0

0100000001f1dddc18fe1c664bbd221d2d18e147f50776cb9aebb0535db7a96c561e0e93e3000000006a473044022003829abee4bff4df3d3ddf0be6f77e311a09b8e945ec33cdf103305914e84f46022043745996a4dcfd6d320378786337c320c936281218803573be66a12881c2ad350121021cdd761c7eb1c90c0af0a5963e94bf0203176b4662778d32bd6d7ab5d8628b32ffffffff01a1629ef5000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
sending: tx: 0100000001f1dddc18fe1c664bbd221d2d18e147f50776cb9aebb0535db7a96c561e0e93e3000000006a473044022003829abee4bff4df3d3ddf0be6f77e311a09b8e945ec33cdf103305914e84f46022043745996a4dcfd6d320378786337c320c936281218803573be66a12881c2ad350121021cdd761c7eb1c90c0af0a5963e94bf0203176b4662778d32bd6d7ab5d8628b32ffffffff01a1629ef5000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788ac00000000
sending: getdata: 0101000000ff821ad1e31f23a4c3f00a74ef3f7f7eca14750c48d001f9f7d0395670f16f61
receiving: merkleblock: 00000020c1000b083ff7e57788d0fef9863d40c1526a16c208aa4a7fbf9e1f5c000000006dce60a5ef240775768983b4da706294